In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


Eu tenho um arquivo secret.py com uma informação sigilosa. Neste caso é uma chave.
Eu vou chamar o conteudo dessa chave usando o código abaixo.


In [3]:
from secret import IEX_CLOUD_API_TOKEN

## Fazer a primeira chamada a API

A api que eu estou usando

A documentação da API que eu estou usando se encontra aqui: https://iexcloud.io/docs/api/


Eu vou buscar na API duas informações importantes:
O market capitalization para cada ação e o preço de cada ação.

Abaixo eu vou usar fstream para passar o simbolo para o endereço.

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
# print(data.status_code)
# Para eu converter o dado recebido na request em json eu só preciso
# Usar o comando abaixo.
data = requests.get(api_url).json()
print(data)


{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'A SLESAQG/LDN NRBLEOT(GTCKSAA )ME', 'calculationPrice': 'iexlasttrade', 'open': None, 'openTime': None, 'openSource': 'oiailffc', 'close': None, 'closeTime': None, 'closeSource': 'lcofiaif', 'high': None, 'highTime': 1640246690517, 'highSource': 'tericEIi Xrlpeae  m', 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 138.48, 'latestSource': 'IEX Last Trade', 'latestTime': 'February 8, 2021', 'latestUpdate': 1657569889888, 'latestVolume': None, 'iexRealtimePrice': 140.72, 'iexRealtimeSize': 21, 'iexLastUpdated': 1677491107283, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 139.64, 'previousVolume': 78878820, 'change': 0.17, 'changePercent': 0.00127, 'volume': None, 'iexMarketPercent': 0.011937064080021445, 'iexVolume': 856539, 

### Um pouco sobre fstrem

Eu uso fstream para modificar alguma tag.

Por exemplo no código abaixo, eu vou trocar a tag {adjective} pela variável adjective acima.

In [5]:
adjective = "superb"
string = f"freeCodeCamp is {adjective}"
print(string)

freeCodeCamp is superb


Parei em 37:45

## Fazer parser do resultado que eu peguei na request da API

In [6]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap)

2403743509427


## Adicionar as informações das ações em um Pandas DataFrame

Para eu passar as informações para um Pandas DataFrame eu preciso antes formatar o DataFrame

In [7]:
# Criar as colunas do dataframe
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


Agora que eu criei o DataFrame, eu vou fazer um append das minhas informações no DataFrame que eu criei logo acima.

Para eu poder adicionar uma informação no meu dataframe eu preciso converter a minha informação em um Panda series. então, se eu tiver um vetor em python que eu quero fazer um append em um dataframe, antes eu preciso converter o vetor em um `pandas series`. 

In [8]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,138.48,2403743509427,N/A


## Adicionando todas as informações das ações no DataFrame usando loop

Eu tenho uma lista com todos as açções. então eu vou usar a minha lista para passar o nome de cada ação para o endpoint que eu pego as informaçoes da ação na minha API.

O código abaixo vai executar muito demorado. Pois ele vai fazer uma request por vez.
Depois eu vou fazer uma forma que eu faça todos os request em batch de uma única só vez.
Pra não demorar tanto eu vou fazer o append apenas das 5 primeiras ações da minha lista de 500 ações.

In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
    index = my_columns),
    ignore_index = True)

In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,123.689,39418960353,N/A
1,AAL,18.520,10965465435,N/A
2,AAP,165.490,10783448378,N/A
3,AAPL,142.140,2373195082129,N/A
4,ABBV,108.970,195477688120,N/A


## Utilizando Batch API Calls para melhorar a performace 

Primeiro eu preciso fazer um split da minha lista em sublistas.

In [11]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [21]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
# symbol_strings vai ser uma lista com todos os nomes de todas as ações
# que eu estou trabalhando. 
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

# Criação de um dataFrame vazio, apenas com a primeira linha que são
#as colunas
final_dataframe = pd.DataFrame(columns = my_columns)

# Fazer uma request em batches. Acontece que aqui eu vou ter uma lista com
# diversos simbolos e eu vou fazer uma request de todos os simbolos 
# de uma única vez
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index = my_columns),
                ignore_index=True
        )
print(final_dataframe)


    Ticker  Stock Price Market Capitalization Number of Shares to Buy
0        A      126.199           38281328926                     N/A
1      AAL       18.100           10800139118                     N/A
2      AAP      161.080           11231893929                     N/A
3     AAPL      139.180         2410903540141                     N/A
4     ABBV      108.270          193516222938                     N/A
..     ...          ...                   ...                     ...
500    YUM      107.630           31316773690                     N/A
501    ZBH      160.920           33261661695                     N/A
502   ZBRA      439.440           23396101315                     N/A
503   ZION       50.600            8250964372                     N/A
504    ZTS      165.130           76564922469                     N/A

[505 rows x 4 columns]


## Calculating the Number of Shares to Buy

Aqui eu vou calcular a quantidade de shares (ações) de cada ação/empresa (stock) que eu quero comprar.

In [26]:
portfolio_size = input('Enter the value of your portfolio: ')

try:
    val = float(portfolio_size)
except ValueError:
    print("O que você colocou não é um número!\nPor favor tente novamente:")
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

Enter the value of your portfolio: 1000000


In [28]:
# position_size é o valor que vai ser dedicado para cada ação.
# Colocar um valor a partir de 1000000 no portfolio_size. Pois essas
# Ações são muito caras e são 505 ações. Menos que isso não dá pra aplicar
# em todas as ações desse conjunto.
position_size = val/len(final_dataframe.index)
#final_dataframe.index - quantidade de linhas de um dataframe
# final_dataframe.loc[i, 'Number of Shares to Buy'] - maneira de você
# ter acesso a um elemento do seu dataframe
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,126.199,38281328926,15
1,AAL,18.100,10800139118,109
2,AAP,161.080,11231893929,12
3,AAPL,139.180,2410903540141,14
4,ABBV,108.270,193516222938,18
...,...,...,...,...
500,YUM,107.630,31316773690,18
501,ZBH,160.920,33261661695,12
502,ZBRA,439.440,23396101315,4
503,ZION,50.600,8250964372,39


Terminei em 1:22:00